# Recurrent Neural Networks  실습

- 실습환경 :  docker + tensorflow + jupyter 
- docker run -i -t  -p 8888:8888  -p 6006:6006  --name tensorflow   gcr.io/tensorflow/tensorflow:latest-devel  /run_jupyter.sh 
- RNN 이론 : https://github.com/biospin/BigBio/blob/master/part02/week04_160426/tensorflow/RNN_%EC%9D%B4%EB%A1%A0.ipynb

- RNN 응용 : [악명생성기, 송치성님](http://nbviewer.jupyter.org/github/daydrill/BadCmtGenerator/blob/master/bad_cmt_generator_code.ipynb)

## Tensorflow가 정상동작 확인 및 예제 소스/데이터  받아오기

In [1]:
import tensorflow as tf

In [5]:
# RNN 예제 위치 확인
! ls -l  /tensorflow/tensorflow/models/rnn/ptb

total 28
-rw-r--r-- 1 root root 1178 Apr 20 20:00 BUILD
-rw-r--r-- 1 root root  892 Apr 20 20:00 __init__.py
-rw-r--r-- 1 root root 9957 Apr 20 20:00 ptb_word_lm.py
-rw-r--r-- 1 root root 3604 Apr 20 20:00 reader.py
-rw-r--r-- 1 root root 2024 Apr 20 20:00 reader_test.py


In [ ]:
! apt-get update 

In [ ]:
! apt-get install wget 

In [10]:
# 학습용 데이터 다운로드
! wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz

--2016-04-23 12:23:55--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)... 147.229.9.23, 2001:67c:1220:809::93e5:917
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34869662 (33M) [application/x-gtar]
Saving to: 'simple-examples.tgz'

100%[======================================>] 34,869,662   948KB/s   in 40s    

2016-04-23 12:24:37 (849 KB/s) - 'simple-examples.tgz' saved [34869662/34869662]



In [ ]:
! tar xvf simple-examples.tgz

## 학습데이터 설명

- Penn Tree Bank (PTB) dataset : Language 모델의 성능평가용으로 유명함.
- 이 dataset은 이미 전처리됨, 1만개 이상의 단어로 구성, marker로 문장의 끝은 표시, 빈도가 낮은 단어는 <unk>로 표시
- reader.py 로 단어들을 유니크한 정수 식별자로 변환함.

## LSTM 관련 코드 설명

- Memory state(gate value)은 zero 벡터로 초기화되고, 단어들을 읽어와서 학습한후에 업데이터됨.
- mini-batches의 크기는 batch_size 만큼 데이터를 처리함.

In [ ]:
lstm = rnn_cell.BasicLSTMCell(lstm_size)
# Initial state of the LSTM memory.
state = tf.zeros([batch_size, lstm.state_size])

loss = 0.0
for current_batch_of_words in words_in_dataset:
    # The value of state is updated after processing each batch of words.
    output, state = lstm(current_batch_of_words, state)

    # The LSTM output can be used to make next word predictions
    logits = tf.matmul(output, softmax_w) + softmax_b
    probabilities = tf.nn.softmax(logits)
    loss += loss_function(probabilities, target_words)

## Backpropagation 관련 코드 설명

- num_steps 만큼 학습되는 단계별로 상태를 확인할 수 있도록 words 변수를 만들어서 gradients값을 저장함.

In [ ]:
# Placeholder for the inputs in a given iteration.
words = tf.placeholder(tf.int32, [batch_size, num_steps])

lstm = rnn_cell.BasicLSTMCell(lstm_size)
# Initial state of the LSTM memory.
initial_state = state = tf.zeros([batch_size, lstm.state_size])

for i in range(num_steps):
    # The value of state is updated after processing each batch of words.
    output, state = lstm(words[:, i], state)

    # The rest of the code.
    # ...

final_state = state

## Input 곤련 코드 설명

- 단어ID들은 LSTM에 사용되기 전에 dense representation으로 embedding됨.[(Vector Representations Tutorial, Word2Vec)](https://www.tensorflow.org/versions/r0.8/tutorials/word2vec/index.html)

In [ ]:
# embedding_matrix is a tensor of shape [vocabulary_size, embedding size]
word_embeddings = tf.nn.embedding_lookup(embedding_matrix, word_ids)

## Loss Function 대한 설명

- 손실함수는 negative log probability의 평균을 최소를 하도록 함.
### $ \text{loss} = -\frac{1}{N}\sum_{i=1}^{N} \ln p_{\text{target}_i}  $

## Stacking multiple LSTMs 관련 소스 설명

- 모델에게 더 많은 표현력을 제공하기 위해, 데이터를 처리하는 LSTMs의 여러 레이어를 추가할 수 있음.
- 첫번째 층의 출력값은 두번째 층의 입력값이 됨.
- **MultiRNNCell** 을 사용하면 됨.

In [ ]:
lstm = rnn_cell.BasicLSTMCell(lstm_size)
stacked_lstm = rnn_cell.MultiRNNCell([lstm] * number_of_layers)

initial_state = state = stacked_lstm.zero_state(batch_size, tf.float32)
for i in range(num_steps):
    # The value of state is updated after processing each batch of words.
    output, state = stacked_lstm(words[:, i], state)

    # The rest of the code.
    # ...

final_state = state

## 코드를 돌려보자.

In [13]:
! python /tensorflow/tensorflow/models/rnn/ptb/ptb_word_lm.py --data_path=./simple-examples/data/ --model small

Epoch: 1 Learning rate: 1.000
0.004 perplexity: 5293.987 speed: 504 wps
0.104 perplexity: 832.714 speed: 516 wps
0.204 perplexity: 618.270 speed: 518 wps
0.304 perplexity: 499.248 speed: 517 wps
0.404 perplexity: 430.411 speed: 511 wps
0.504 perplexity: 385.808 speed: 511 wps
0.604 perplexity: 347.781 speed: 511 wps
0.703 perplexity: 321.963 speed: 511 wps
0.803 perplexity: 301.378 speed: 507 wps
0.903 perplexity: 282.505 speed: 506 wps
Epoch: 1 Train Perplexity: 268.281
Epoch: 1 Valid Perplexity: 180.144
Epoch: 2 Learning rate: 1.000
0.004 perplexity: 213.310 speed: 477 wps
0.104 perplexity: 151.881 speed: 474 wps
0.204 perplexity: 159.060 speed: 488 wps
0.304 perplexity: 154.072 speed: 488 wps
0.404 perplexity: 150.989 speed: 494 wps
0.504 perplexity: 148.517 speed: 497 wps
0.604 perplexity: 143.812 speed: 497 wps
0.703 perplexity: 141.676 speed: 499 wps
0.803 perplexity: 139.689 speed: 500 wps
0.903 perplexity: 135.973 speed: 501 wps
Epoch: 2 Train Perplexity: 133.827
Epoch: 2 Valid

! cat /notebook/tensorflow/tensorflow/models/rnn/ptb/ptb_word_lm.py

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import numpy as np
import tensorflow as tf

from tensorflow.models.rnn.ptb import reader

flags = tf.flags
logging = tf.logging

flags.DEFINE_string(
    "model", "small",
    "A type of model. Possible options are: small, medium, large.")
flags.DEFINE_string("data_path", "/notebook/simple-examples/data/", "data_path")

FLAGS = flags.FLAGS


class PTBModel(object):
  """The PTB model."""

  def __init__(self, is_training, config):
    self.batch_size = batch_size = config.batch_size
    self.num_steps = num_steps = config.num_steps
    size = config.hidden_size
    vocab_size = config.vocab_size

    self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
    self._targets = tf.placeholder(tf.int32, [batch_size, num_steps])

    # Slightly better results can be obtained with forget gate biases
    # initialized to 1 but the hyperparameters of the model would need to be
    # different than reported in the paper.
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0)
    if is_training and config.keep_prob < 1:
      lstm_cell = tf.nn.rnn_cell.DropoutWrapper(
          lstm_cell, output_keep_prob=config.keep_prob)
    cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * config.num_layers)

    self._initial_state = cell.zero_state(batch_size, tf.float32)

    with tf.device("/cpu:0"):
      embedding = tf.get_variable("embedding", [vocab_size, size])
      inputs = tf.nn.embedding_lookup(embedding, self._input_data)

    if is_training and config.keep_prob < 1:
      inputs = tf.nn.dropout(inputs, config.keep_prob)

    # Simplified version of tensorflow.models.rnn.rnn.py's rnn().
    # This builds an unrolled LSTM for tutorial purposes only.
    # In general, use the rnn() or state_saving_rnn() from rnn.py.
    #
    # The alternative version of the code below is:
    #
    # from tensorflow.models.rnn import rnn
    # inputs = [tf.squeeze(input_, [1])
    #           for input_ in tf.split(1, num_steps, inputs)]
    # outputs, state = rnn.rnn(cell, inputs, initial_state=self._initial_state)
    outputs = []
    state = self._initial_state
    with tf.variable_scope("RNN"):
      for time_step in range(num_steps):
        if time_step > 0: tf.get_variable_scope().reuse_variables()
        (cell_output, state) = cell(inputs[:, time_step, :], state)
        outputs.append(cell_output)

    output = tf.reshape(tf.concat(1, outputs), [-1, size])
    softmax_w = tf.get_variable("softmax_w", [size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    logits = tf.matmul(output, softmax_w) + softmax_b
    loss = tf.nn.seq2seq.sequence_loss_by_example(
        [logits],
        [tf.reshape(self._targets, [-1])],
        [tf.ones([batch_size * num_steps])])
    self._cost = cost = tf.reduce_sum(loss) / batch_size
    self._final_state = state

    if not is_training:
      return

    self._lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                      config.max_grad_norm)
    optimizer = tf.train.GradientDescentOptimizer(self.lr)
    self._train_op = optimizer.apply_gradients(zip(grads, tvars))

  def assign_lr(self, session, lr_value):
    session.run(tf.assign(self.lr, lr_value))

  @property
  def input_data(self):
    return self._input_data

  @property
  def targets(self):
    return self._targets

  @property
  def initial_state(self):
    return self._initial_state

  @property
  def cost(self):
    return self._cost

  @property
  def final_state(self):
    return self._final_state

  @property
  def lr(self):
    return self._lr

  @property
  def train_op(self):
    return self._train_op


class SmallConfig(object):
  """Small config."""
  init_scale = 0.1
  learning_rate = 1.0
  max_grad_norm = 5
  num_layers = 2
  num_steps = 20
  hidden_size = 200
  max_epoch = 4
  max_max_epoch = 13
  keep_prob = 1.0
  lr_decay = 0.5
  batch_size = 20
  vocab_size = 10000


class MediumConfig(object):
  """Medium config."""
  init_scale = 0.05
  learning_rate = 1.0
  max_grad_norm = 5
  num_layers = 2
  num_steps = 35
  hidden_size = 650
  max_epoch = 6
  max_max_epoch = 39
  keep_prob = 0.5
  lr_decay = 0.8
  batch_size = 20
  vocab_size = 10000


class LargeConfig(object):
  """Large config."""
  init_scale = 0.04
  learning_rate = 1.0
  max_grad_norm = 10
  num_layers = 2
  num_steps = 35
  hidden_size = 1500
  max_epoch = 14
  max_max_epoch = 55
  keep_prob = 0.35
  lr_decay = 1 / 1.15
  batch_size = 20
  vocab_size = 10000


class TestConfig(object):
  """Tiny config, for testing."""
  init_scale = 0.1
  learning_rate = 1.0
  max_grad_norm = 1
  num_layers = 1
  num_steps = 2
  hidden_size = 2
  max_epoch = 1
  max_max_epoch = 1
  keep_prob = 1.0
  lr_decay = 0.5
  batch_size = 20
  vocab_size = 10000


def run_epoch(session, m, data, eval_op, verbose=False):
  """Runs the model on the given data."""
  epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
  start_time = time.time()
  costs = 0.0
  iters = 0
  state = m.initial_state.eval()
  for step, (x, y) in enumerate(reader.ptb_iterator(data, m.batch_size,
                                                    m.num_steps)):
    cost, state, _ = session.run([m.cost, m.final_state, eval_op],
                                 {m.input_data: x,
                                  m.targets: y,
                                  m.initial_state: state})
    costs += cost
    iters += m.num_steps

    if verbose and step % (epoch_size // 10) == 10:
      print("%.3f perplexity: %.3f speed: %.0f wps" %
            (step * 1.0 / epoch_size, np.exp(costs / iters),
             iters * m.batch_size / (time.time() - start_time)))

  return np.exp(costs / iters)


def get_config():
  if FLAGS.model == "small":
    return SmallConfig()
  elif FLAGS.model == "medium":
    return MediumConfig()
  elif FLAGS.model == "large":
    return LargeConfig()
  elif FLAGS.model == "test":
    return TestConfig()
  else:
    raise ValueError("Invalid model: %s", FLAGS.model)


def main(_):
  if not FLAGS.data_path:
    raise ValueError("Must set --data_path to PTB data directory")

  raw_data = reader.ptb_raw_data(FLAGS.data_path)
  train_data, valid_data, test_data, _ = raw_data

  config = get_config()
  eval_config = get_config()
  eval_config.batch_size = 1
  eval_config.num_steps = 1

  with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-config.init_scale,
                                                config.init_scale)
    with tf.variable_scope("model", reuse=None, initializer=initializer):
      m = PTBModel(is_training=True, config=config)
    with tf.variable_scope("model", reuse=True, initializer=initializer):
      mvalid = PTBModel(is_training=False, config=config)
      mtest = PTBModel(is_training=False, config=eval_config)

    tf.initialize_all_variables().run()

    for i in range(config.max_max_epoch):
      lr_decay = config.lr_decay ** max(i - config.max_epoch, 0.0)
      m.assign_lr(session, config.learning_rate * lr_decay)

      print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
      train_perplexity = run_epoch(session, m, train_data, m.train_op,
                                   verbose=True)
      print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
      valid_perplexity = run_epoch(session, mvalid, valid_data, tf.no_op())
      print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

    test_perplexity = run_epoch(session, mtest, test_data, tf.no_op())
    print("Test Perplexity: %.3f" % test_perplexity)


if __name__ == "__main__":
  tf.app.run()